In [166]:
# !pip install split-folders

In [168]:
# import splitfolders
# splitfolders.ratio("D:\\Stroke Data", output="D:\\Stroke Data_Splitted\\",seed=1337, ratio=(.7, .3), group_prefix=None, move=False)

In [51]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, MultiHeadAttention, LayerNormalization, Dense, Dropout
from sklearn.metrics import mean_squared_error

# Load your datasets
Straight_Drive_df = pd.read_csv("Straight_Drive.csv")
Sweep_df = pd.read_csv("Sweep.csv")
Pull_df = pd.read_csv("Pull.csv")
onDrive_df = pd.read_csv("onDrive.csv")
Flick_df = pd.read_csv("Flick.csv")
Cut_df = pd.read_csv("Cut.csv")
Cover_Drive_df = pd.read_csv("Cover_Drive.csv")
back_foot_punch_df = pd.read_csv("back_foot_punch.csv")

# Using 'Cover_Drive.csv' dataset
Posdataset = pd.read_csv('Cover_Drive.csv')
# Drop the non-numeric columns ('Unnamed: 0' and 'Label')
Posdataset_numeric = Posdataset.drop(columns=['Unnamed: 0', 'Label'], axis=1)

In [53]:
Posdataset_numeric.head(5)

,F0_x,F0_y,F0_z,F0_vis,F1_x,F1_y,F1_z,F1_vis,F2_x,F2_y,...,F30_z,F30_vis,F31_x,F31_y,F31_z,F31_vis,F32_x,F32_y,F32_z,F32_vis
0,0.605604,0.319193,-0.206423,0.998100,0.608060,0.310672,-0.205325,0.997993,0.609513,0.310109,...,0.052371,0.704735,0.636576,0.643802,-0.210230,0.918661,0.629231,0.632027,0.011629,0.861975
1,0.605210,0.320216,-0.195480,0.998275,0.607313,0.310882,-0.193891,0.998183,0.608708,0.310283,...,-0.000988,0.693936,0.637445,0.648802,-0.216752,0.922947,0.629856,0.631912,-0.047416,0.844693
2,0.605156,0.321100,-0.198475,0.998430,0.607107,0.311066,-0.197096,0.998352,0.608519,0.310419,...,0.006449,0.685689,0.637536,0.646330,-0.215665,0.927065,0.630199,0.631907,-0.039618,0.830709
3,0.602378,0.320967,-0.149304,0.998564,0.604508,0.311048,-0.147209,0.998500,0.605890,0.310358,...,0.022291,0.679551,0.637206,0.644141,-0.173419,0.930712,0.631187,0.631652,-0.020749,0.814591
4,0.600436,0.320933,-0.092031,0.998703,0.602337,0.311165,-0.096430,0.998647,0.603642,0.310417,...,0.083066,0.685475,0.637198,0.642320,-0.159816,0.936338,0.632738,0.629347,0.058007,0.807283


Let's break this code down step by step and explain with an example using **numeric values**.

### Problem Context
- We have a dataset `Posdataset_numeric`, where each row corresponds to a "frame" (e.g., keypoints of a cricket stroke).
- Each frame has `n_features = 4` features (which could represent coordinates or other numeric data).
- We're using a sliding window of `timesteps = 5` to predict the next frame.
- We'll train our model using the first 25 frames and test it on the next 7 frames.

### Key Concepts

- **Timesteps**: The number of past frames (i.e., 5) used to predict the next one.
- **Training Set**: The first 25 frames used for training.
- **Test Set**: The next 7 frames used to evaluate the model.

### Example Data
Assume our dataset (`Posdataset_numeric`) looks like this (simplified with random values):

```
Frame | Feature1 | Feature2 | Feature3 | Feature4
---------------------------------------------------
1     | 0.1      | 0.2      | 0.3      | 0.4
2     | 0.5      | 0.6      | 0.7      | 0.8
3     | 0.9      | 1.0      | 1.1      | 1.2
4     | 1.3      | 1.4      | 1.5      | 1.6
5     | 1.7      | 1.8      | 1.9      | 2.0
6     | 2.1      | 2.2      | 2.3      | 2.4
7     | 2.5      | 2.6      | 2.7      | 2.8
...
32    | 9.7      | 9.8      | 9.9      | 10.0
```

### Data Splitting
1. **Training Set** (`train_size = 25`): This includes the first 25 frames (rows 1 to 25).
2. **Test Set** (`test_size = 7`): This includes the next 7 frames (rows 26 to 32).

```python
train_data = Posdataset_numeric.iloc[:25].values  # First 25 rows
test_data = Posdataset_numeric.iloc[25:32].values # Next 7 rows
```

### Sequence Creation
The function `create_sequences` generates sequences (subsets of the data) for input (X) and target output (y).

- **X**: Contains 5 consecutive frames (as defined by `timesteps = 5`).
- **y**: Contains the next frame to predict (i.e., the frame that comes after the 5th frame in the sequence).

#### Training Set Example:
Let's consider the **first sequence** generated from the training set (`train_data`).

- **X** (first sequence): Frames 1 to 5 (each frame has 4 features):
  ```
  [[0.1, 0.2, 0.3, 0.4], 
   [0.5, 0.6, 0.7, 0.8], 
   [0.9, 1.0, 1.1, 1.2], 
   [1.3, 1.4, 1.5, 1.6], 
   [1.7, 1.8, 1.9, 2.0]]
  ```
- **y** (first target): Frame 6 (the next frame):
  ```
  [2.1, 2.2, 2.3, 2.4]
  ```

The function repeats this process, sliding the window over the data to create additional sequences.

#### Final Shape:
After generating all possible sequences in the training data:
- **X_train shape**: `(20, 5, 4)` → 20 sequences, each containing 5 frames, with 4 features per frame.
- **y_train shape**: `(20, 4)` → 20 target frames, each with 4 features.

#### Test Set Example:
For the test data (`test_data`):
- The same sliding window approach is applied, using the first 5 test frames to predict the 6th one, and so on.

### Code Walkthrough:

```python
# Parameters
timesteps = 5  # Use 5 previous frames for prediction
n_features = 4  # Each frame has 4 features
train_size = 25
test_size = 7

# Split the data into training and test sets
train_data = Posdataset_numeric.iloc[:train_size].values  # First 25 frames
test_data = Posdataset_numeric.iloc[train_size:train_size + test_size].values  # Next 7 frames

# Function to create sequences (X as input, y as target)
def create_sequences(data, timesteps):
    X, y = [], []
    for i in range(len(data) - timesteps):
        X.append(data[i:i + timesteps])  # 5 frames as input
        y.append(data[i + timesteps])    # The next frame as output
    return np.array(X), np.array(y)

# Create sequences for training
X_train, y_train = create_sequences(train_data, timesteps)
# Create sequences for testing
X_test, y_test = create_sequences(test_data, timesteps)

# Check the shapes of the resulting arrays
print('X_Train Data Shape: ', X_train.shape)  # (20, 5, 4)
print('y_Train Data Shape: ', y_train.shape)  # (20, 4)
print('X_Test Data Shape: ', X_test.shape)    # (2, 5, 4)
print('y_Test Data Shape: ', y_test.shape)    # (2, 4)
```

### Explanation of Shapes:
- **X_train shape**: `(20, 5, 4)` means there are 20 sequences, each consisting of 5 frames, with 4 features per frame.
- **y_train shape**: `(20, 4)` means there are 20 target frames, each having 4 features (corresponding to the next frame after each sequence).
- **X_test shape**: `(2, 5, 4)` means there are 2 sequences in the test set
32), each consisting of 5 frames.
- **y_test shape**: `(2, 4)` means there are 2 target frames in the test set.

In this example, you are predicting the next frame in a sequence of frames (like predicting the next move in a cricket stroke based on past moves).

### Predict the next 3 frames
To modify the code to **predict the next 3 frames** instead of just the next one, you need to adjust the `create_sequences` function to generate **multiple future frames** as targets.

Here's how you can update the code:

### Changes Required:

1. **X** (input) will remain the same: The past `timesteps` number of frames.
2. **y** (target) will now be the next **3 consecutive frames** after the `timesteps` number of frames.

### Updated `create_sequences` Function:

- Instead of appending only one frame as the target (`y.append(data[i + timesteps])`), you will append the next 3 frames (`y.append(data[i + timesteps:i + timesteps + 3])`).
- This will ensure that for every input sequence, there are 3 future frames as the target.

### Code Update:

```python
# Parameters
timesteps = 5  # Use 5 previous frames for prediction
n_features = 4  # Each frame has 4 features
train_size = 25
test_size = 7
n_future_frames = 3  # Predict the next 3 frames

# Split the data into training and test sets
train_data = Posdataset_numeric.iloc[:train_size].values  # First 25 frames
test_data = Posdataset_numeric.iloc[train_size:train_size + test_size].values  # Next 7 frames

# Function to create sequences (X as input, y as target with 3 future frames)
def create_sequences(data, timesteps, n_future_frames):
    X, y = [], []
    for i in range(len(data) - timesteps - n_future_frames + 1):
        X.append(data[i:i + timesteps])  # 5 frames as input
        y.append(data[i + timesteps:i + timesteps + n_future_frames])  # Next 3 frames as output
    return np.array(X), np.array(y)

# Create sequences for training
X_train, y_train = create_sequences(train_data, timesteps, n_future_frames)
# Create sequences for testing
X_test, y_test = create_sequences(test_data, timesteps, n_future_frames)

# Check the shapes of the resulting arrays
print('X_Train Data Shape: ', X_train.shape)  # Shape: (number of sequences, 5, 4)
print('y_Train Data Shape: ', y_train.shape)  # Shape: (number of sequences, 3, 4)
print('X_Test Data Shape: ', X_test.shape)    # Shape: (number of sequences, 5, 4)
print('y_Test Data Shape: ', y_test.shape)    # Shape: (number of sequences, 3, 4)
```

### Explanation of Changes:

1. **X (input)** remains a sequence of 5 frames (with `timesteps = 5`).
2. **y (target)** now contains 3 future frames instead of just one.
   - So, for every sequence of 5 frames in `X`, the target `y` will be the next 3 frames (each having 4 features).
3. The sliding window logic is updated so that you only generate sequences where 3 future frames exist after the current sequence.

### Example:

Assume we have the following data (simplified example with random values):

```
Frame | Feature1 | Feature2 | Feature3 | Feature4
---------------------------------------------------
1     | 0.1      | 0.2      | 0.3      | 0.4
2     | 0.5      | 0.6      | 0.7      | 0.8
3     | 0.9      | 1.0      | 1.1      | 1.2
4     | 1.3      | 1.4      | 1.5      | 1.6
5     | 1.7      | 1.8      | 1.9      | 2.0
6     | 2.1      | 2.2      | 2.3      | 2.4
7     | 2.5      | 2.6      | 2.7      | 2.8
8     | 2.9      | 3.0      | 3.1      | 3.2
9     | 3.3      | 3.4      | 3.5      | 3.6
...
```

For the first sequence:
- **X** (input) will be frames 1 to 5:
  ```
  [[0.1, 0.2, 0.3, 0.4], 
   [0.5, 0.6, 0.7, 0.8], 
   [0.9, 1.0, 1.1, 1.2], 
   [1.3, 1.4, 1.5, 1.6], 
   [1.7, 1.8, 1.9, 2.0]]
  ```
- **y** (target) will be frames 6 to 8 (3 future frames):
  ```
  [[2.1, 2.2, 2.3, 2.4], 
   [2.5, 2.6, 2.7, 2.8], 
   [2.9, 3.0, 3.1, 3.2]]
  ```

### Shape of Output:

1. **X_train shape**: `(number_of_sequences, timesteps, n_features)`
   - Example: `(18, 5, 4)` → There are 18 sequences, each with 5 frames, and each frame has 4 features.
2. **y_train shape**: `(number_of_sequences, n_future_frames, n_features)`
   - Example: `(18, 3, 4)` → There are 18 sequences, each predicting 3 future frames, and each frame has 4 features.

By predicting 3 future frames, the model can forecast more extended sequences based on the past frames, which is useful in scenarios like time-series forecasting or predicting movements in sports based on previous patterns.

In [90]:
# Parameters
timesteps = 10  # Number of previous frames to use for prediction
n_features = Posdataset_numeric.shape[1]  # Number of features
train_size = len(Posdataset_numeric)-11  # Train on first 830 frames
test_size = 11   # Test on the next 10 frames (831 to 840)

#print(len(Posdataset_numeric))
# Split the data into train and test sets
train_data = Posdataset_numeric.iloc[:train_size].values
test_data = Posdataset_numeric.iloc[train_size:train_size + test_size].values

# Scale the data
scaler = MinMaxScaler()
train_data_scaled = scaler.fit_transform(train_data)
test_data_scaled = scaler.transform(test_data)
print('TrainData Shape: ', train_data_scaled.shape)
print('TestData Shape: ',test_data_scaled.shape)


TrainData Shape:  (832, 132)
TestData Shape:  (11, 132)


In [93]:
# Function to create sequences
def create_sequences(data, timesteps):
    X, y = [], []
    for i in range(len(data) - timesteps):
        X.append(data[i:i + timesteps])
        y.append(data[i + timesteps])
    return np.array(X), np.array(y)

# Create sequences from the scaled data
X_train, y_train = create_sequences(train_data_scaled, timesteps)
X_test, y_test = create_sequences(test_data_scaled, timesteps)

print('X_Train Data Shape: ', X_train.shape)
print('y_Train Data Shape: ', y_train.shape)
print('X_Test Data Shape: ',X_test.shape)
print('y_Test Data Shape: ',y_test.shape)

X_Train Data Shape:  (822, 10, 132)
y_Train Data Shape:  (822, 132)
X_Test Data Shape:  (1, 10, 132)
y_Test Data Shape:  (1, 132)


In [97]:
# Reshape y_train and y_test to include the time dimension
y_train = y_train.reshape(y_train.shape[0], 1, y_train.shape[1])  # Shape: (820, 1, n_features)
y_test = y_test.reshape(y_test.shape[0], 1, y_test.shape[1])      # Shape: (1, 1, n_features)


# Check the shapes to ensure they are correct
print("y_train shape:", y_train.shape)  # Should be (samples, 1, n_features)
print("y_test shape:", y_test.shape)    # Should be (samples, 1, n_features)



y_train shape: (822, 1, 132)
y_test shape: (1, 1, 132)


In [99]:
# Function to create and train Transformer model
def build_transformer_model():
    input_layer = Input(shape=(timesteps, n_features))
    attention_output = MultiHeadAttention(num_heads=4, key_dim=n_features)(input_layer, input_layer)
    attention_output = LayerNormalization(epsilon=1e-6)(attention_output)
    transformer_output = Dense(64, activation='relu')(attention_output)
    transformer_output = Dropout(0.2)(transformer_output)
    output = Dense(n_features)(transformer_output)

    model = Model(inputs=input_layer, outputs=output)
    model.compile(optimizer='adam', loss='mse')
    return model

# Train and evaluate the model
model = build_transformer_model()

In [104]:
print(X_test.shape)

(1, 10, 132)


In [106]:
# Number of epochs and batch size
epochs = 20
batch_size = 32

# Fit the model on the training data and validate on the test data
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test), verbose=1)
# Predict the next frames
# Predict the next frames
predictions = model.predict(X_test)

Epoch 1/20
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - loss: 0.0275 - val_loss: 0.0056
Epoch 2/20
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 0.0262 - val_loss: 0.0053
Epoch 3/20
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.0265 - val_loss: 0.0048
Epoch 4/20
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.0259 - val_loss: 0.0043
Epoch 5/20
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.0256 - val_loss: 0.0038
Epoch 6/20
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - loss: 0.0257 - val_loss: 0.0055
Epoch 7/20
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.0250 - val_loss: 0.0049
Epoch 8/20
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 0.0240 - val_loss: 0.0041
Epoch 9/20
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 0.0242 - val_loss: 0.0038
Epoch 10/20
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 0.0233 - val_loss: 0.0042
Epoch 11/20
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.0237 - val_loss: 0.0056
Epoch 12/20
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.0

In [108]:
# Check the shapes of predictions and y_test before reshaping
print("Predictions shape:", predictions.shape)
print("y_test shape before reshaping:", y_test.shape)

Predictions shape: (1, 10, 132)
y_test shape before reshaping: (1, 1, 132)


In [133]:
# Reshape predictions by flattening time steps into the batch dimension
#predictions_reshaped = predictions.reshape(predictions.shape[0], -1)
# Take the last time step's predictions (for example, if you're interested in the last time step)
predictions_reshaped = predictions[:, -1, :]
y_test_reshaped = y_test[:, -1, :]

print("Predictions reshaped shape:", predictions_reshaped.shape)
print("y_test_reshaped:", y_test_reshaped.shape)
# Rescale predictions back to original values
predictions_rescaled = scaler.inverse_transform(predictions_reshaped)
y_test_rescaled = scaler.inverse_transform(y_test_reshaped)
print("y_test_rescaled:", y_test_rescaled.shape, type(y_test_rescaled))
# Calculate error
mse = mean_squared_error(y_test_rescaled, predictions_rescaled)
print(f'Transformer Model MSE: {mse}')



Predictions reshaped shape: (1, 132)
y_test_reshaped: (1, 132)
y_test_rescaled: (1, 132) <class 'numpy.ndarray'>
Transformer Model MSE: 0.0007538144151298676


In [127]:
print(type(predictions_rescaled))

<class 'numpy.ndarray'>


In [115]:
# Print predicted vs actual frames for comparison
print("Predicted frames:")
print(predictions_rescaled)
print("Actual frames:")
print(y_test_rescaled)

Predicted frames:
[[ 3.9956656e-01  4.3985844e-01 -1.4939047e-01  9.9968761e-01
   4.0712228e-01  4.3592298e-01 -1.4596486e-01  9.9942440e-01
   4.0762785e-01  4.2930570e-01 -1.3614281e-01  9.9938035e-01
   4.0560079e-01  4.3806753e-01 -1.4055437e-01  9.9959707e-01
   3.9746985e-01  4.3336329e-01 -1.3600844e-01  9.9975044e-01
   3.9656636e-01  4.4129357e-01 -1.4156111e-01  9.9941349e-01
   3.9890176e-01  4.4442809e-01 -1.3635139e-01  9.9941880e-01
   4.1330662e-01  4.3065986e-01 -9.5798507e-02  9.9931043e-01
   3.9893588e-01  4.4109905e-01 -6.1210148e-02  9.9901485e-01
   4.1381541e-01  4.5284590e-01 -1.4021988e-01  9.9946082e-01
   4.0566331e-01  4.5163643e-01 -1.2420874e-01  9.9839884e-01
   4.3571264e-01  4.3908972e-01 -9.0509735e-02  9.9894738e-01
   4.0454918e-01  4.9051833e-01  4.5107477e-04  9.9609143e-01
   4.4806966e-01  3.8597009e-01 -1.5999867e-01  9.8893803e-01
   3.9528346e-01  5.1513433e-01 -5.9583869e-02  9.2917347e-01
   4.0263802e-01  4.0332076e-01 -1.7283225e-01  8.65

In [117]:

#print("Predicted Pose Points for Next Frame:", len(predicted_frame[0]))  # Extract the predicted pose points

print(predictions_rescaled.shape)

(1, 132)


In [123]:
#Posdataset
data = Posdataset.drop(columns=['Unnamed: 0'])
Features=data.drop( ['Label'], axis=1)
#print(Features)
# Assuming 'newdataset' is your DataFrame
columns_list = Features.columns.tolist()

# nt the list of column names
#print(columns_list)
preframes=predictions_rescaled
print('Predicted Frame: ',preframes)
print(len(columns_list))

# Reshape frames to 2D (if it is a 3D array)
preframes = preframes.reshape(1, 132)

# Now create the DataFrame
PredictedFrame = pd.DataFrame(preframes, columns=columns_list)

orgframes = y_test_rescaled
orgframes = orgframes.reshape(1, 132)
OrignalFrame = pd.DataFrame(orgframes, columns=columns_list)



Predicted Frame:  [[ 3.9956656e-01  4.3985844e-01 -1.4939047e-01  9.9968761e-01
   4.0712228e-01  4.3592298e-01 -1.4596486e-01  9.9942440e-01
   4.0762785e-01  4.2930570e-01 -1.3614281e-01  9.9938035e-01
   4.0560079e-01  4.3806753e-01 -1.4055437e-01  9.9959707e-01
   3.9746985e-01  4.3336329e-01 -1.3600844e-01  9.9975044e-01
   3.9656636e-01  4.4129357e-01 -1.4156111e-01  9.9941349e-01
   3.9890176e-01  4.4442809e-01 -1.3635139e-01  9.9941880e-01
   4.1330662e-01  4.3065986e-01 -9.5798507e-02  9.9931043e-01
   3.9893588e-01  4.4109905e-01 -6.1210148e-02  9.9901485e-01
   4.1381541e-01  4.5284590e-01 -1.4021988e-01  9.9946082e-01
   4.0566331e-01  4.5163643e-01 -1.2420874e-01  9.9839884e-01
   4.3571264e-01  4.3908972e-01 -9.0509735e-02  9.9894738e-01
   4.0454918e-01  4.9051833e-01  4.5107477e-04  9.9609143e-01
   4.4806966e-01  3.8597009e-01 -1.5999867e-01  9.8893803e-01
   3.9528346e-01  5.1513433e-01 -5.9583869e-02  9.2917347e-01
   4.0263802e-01  4.0332076e-01 -1.7283225e-01  8.65

In [125]:

import cv2
import pandas as pd
import numpy as np

# Select only the columns that contain x, y, and z coordinates
xyz_columns = [col for col in PredictedFrame.columns if '_x' in col or '_y' in col or '_z' in col]
predicted_xyz = PredictedFrame[xyz_columns]
original_xyz = OrignalFrame[xyz_columns]

# Define connections between keypoints to draw the skeleton based on F1 to F32
connections = [
    (0, 1), (1, 2), (2, 3),  # Left eye connections
    (0, 4), (4, 5), (5, 6),  # Right eye connections
    (0, 7), (0, 8),  # Nose to ears
    (9, 10),  # Mouth connection
    (11, 12),  # Shoulders
    (11, 13), (13, 15),  # Left arm (shoulder, elbow, wrist)
    (15, 17), (15, 19), (15, 21),  # Left hand (wrist to pinky, index, thumb)
    (12, 14), (14, 16),  # Right arm (shoulder, elbow, wrist)
    (16, 18), (16, 20), (16, 22),  # Right hand (wrist to pinky, index, thumb)
    (23, 24),  # Hips
    (11, 23), (12, 24),  # Shoulders to hips
    (23, 25), (25, 27), (27, 29), (27, 31),  # Left leg (hip, knee, ankle, heel, foot index)
    (24, 26), (26, 28), (28, 30), (28, 32),  # Right leg (hip, knee, ankle, heel, foot index)
]

# Create a blank image (adjust size according to your coordinates)
width, height = 640, 480  # Adjust size as needed

# Loop through each row (frame) in the DataFrame to draw the skeleton for each frame
for index in range(len(predicted_xyz)):
    # Clear the image for each frame
    blank_image = np.zeros((height, width, 3), dtype=np.uint8)

    # Extract (x, y) coordinates from the predicted row
    predicted_row = predicted_xyz.iloc[index]
    predicted_keypoints = []
    for i in range(0, len(predicted_row), 3):  # Skip every 3rd value because of the x, y, z format
        x = predicted_row[i] * width  # Assuming x is normalized, scale to image width
        y = predicted_row[i + 1] * height  # Assuming y is normalized, scale to image height
        predicted_keypoints.append((int(x), int(y)))

    # Extract (x, y) coordinates from the original row
    original_row = original_xyz.iloc[index]
    original_keypoints = []
    for i in range(0, len(original_row), 3):  # Skip every 3rd value because of the x, y, z format
        x = original_row[i] * width  # Assuming x is normalized, scale to image width
        y = original_row[i + 1] * height  # Assuming y is normalized, scale to image height
        original_keypoints.append((int(x), int(y)))

    # Draw circles at each keypoint for the predicted frame (color: green)
    for (x, y) in predicted_keypoints:
        cv2.circle(blank_image, (x, y), 5, (0, 255, 0), cv2.FILLED)

    # Draw lines to form the skeleton for the predicted frame (color: blue)
    for connection in connections:
        point1 = connection[0]
        point2 = connection[1]
        if point1 < len(predicted_keypoints) and point2 < len(predicted_keypoints):
            cv2.line(blank_image, predicted_keypoints[point1], predicted_keypoints[point2], (0, 255, 0), 2)

    # Draw circles at each keypoint for the original frame (color: red)
    for (x, y) in original_keypoints:
        cv2.circle(blank_image, (x, y), 5, (0, 0, 255), cv2.FILLED)

    # Draw lines to form the skeleton for the original frame (color: yellow)
    for connection in connections:
        point1 = connection[0]
        point2 = connection[1]
        if point1 < len(original_keypoints) and point2 < len(original_keypoints):
            cv2.line(blank_image, original_keypoints[point1], original_keypoints[point2], (0, 0, 255), 2)

    # Add text to indicate which color represents original and predicted
    cv2.putText(blank_image, 'Original: Pose Red', (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
    cv2.putText(blank_image, 'Predicted: Pose Green', (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
    #cv2.putText(blank_image, 'Original: Shot ' + str(Org_label[0]), (10, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
    #cv2.putText(blank_image, 'Predicted: Shot ' + str(pred_label[0]), (10, 120), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    # Show the skeleton for this frame
    cv2.imshow('Skeleton Comparison', blank_image)
    
    key = cv2.waitKey(50000) & 0xFF  # Adjust wait time to 30 milliseconds
    if key == ord('q'):
        print("Exiting on 'q' key press")
        break

# Release resources
cv2.destroyAllWindows()

import cv2
import pandas as pd
import numpy as np

# Select only the columns that contain x, y, and z coordinates
xyz_columns = [col for col in PredictedFrame.columns if '_x' in col or '_y' in col or '_z' in col]
predicted_xyz = PredictedFrame[xyz_columns]
original_xyz = OrignalFrame[xyz_columns]

# Define connections between keypoints to draw the skeleton based on F1 to F32
connections = [
    (0, 1), (1, 2), (2, 3),  # Left eye connections
    (0, 4), (4, 5), (5, 6),  # Right eye connections
    (0, 7), (0, 8),  # Nose to ears
    (9, 10),  # Mouth connection
    (11, 12),  # Shoulders
    (11, 13), (13, 15),  # Left arm (shoulder, elbow, wrist)
    (15, 17), (15, 19), (15, 21),  # Left hand (wrist to pinky, index, thumb)
    (12, 14), (14, 16),  # Right arm (shoulder, elbow, wrist)
    (16, 18), (16, 20), (16, 22),  # Right hand (wrist to pinky, index, thumb)
    (23, 24),  # Hips
    (11, 23), (12, 24),  # Shoulders to hips
    (23, 25), (25, 27), (27, 29), (27, 31),  # Left leg (hip, knee, ankle, heel, foot index)
    (24, 26), (26, 28), (28, 30), (28, 32),  # Right leg (hip, knee, ankle, heel, foot index)
]

# Create a blank image (adjust size according to your coordinates)
width, height = 640, 480  # Adjust size as needed

# Loop through each row (frame) in the DataFrame to draw the skeleton for each frame
for index in range(len(predicted_xyz)):
    # Clear images for each frame
    blank_image_predicted = np.zeros((height, width, 3), dtype=np.uint8)
    blank_image_original = np.zeros((height, width, 3), dtype=np.uint8)

    # Extract (x, y) coordinates from the predicted row
    predicted_row = predicted_xyz.iloc[index]
    predicted_keypoints = []
    for i in range(0, len(predicted_row), 3):  # Skip every 3rd value because of the x, y, z format
        x = predicted_row[i] * width  # Assuming x is normalized, scale to image width
        y = predicted_row[i + 1] * height  # Assuming y is normalized, scale to image height
        predicted_keypoints.append((int(x), int(y)))

    # Extract (x, y) coordinates from the original row
    original_row = original_xyz.iloc[index]
    original_keypoints = []
    for i in range(0, len(original_row), 3):  # Skip every 3rd value because of the x, y, z format
        x = original_row[i] * width  # Assuming x is normalized, scale to image width
        y = original_row[i + 1] * height  # Assuming y is normalized, scale to image height
        original_keypoints.append((int(x), int(y)))

    # Draw circles and lines for the predicted frame (color: green)
    for (x, y) in predicted_keypoints:
        cv2.circle(blank_image_predicted, (x, y), 5, (0, 255, 0), cv2.FILLED)
    for connection in connections:
        point1, point2 = connection
        if point1 < len(predicted_keypoints) and point2 < len(predicted_keypoints):
            cv2.line(blank_image_predicted, predicted_keypoints[point1], predicted_keypoints[point2], (0, 255, 0), 2)

    # Draw circles and lines for the original frame (color: red)
    for (x, y) in original_keypoints:
        cv2.circle(blank_image_original, (x, y), 5, (0, 0, 255), cv2.FILLED)
    for connection in connections:
        point1, point2 = connection
        if point1 < len(original_keypoints) and point2 < len(original_keypoints):
            cv2.line(blank_image_original, original_keypoints[point1], original_keypoints[point2], (0, 0, 255), 2)

    # Add text for frame labels
    cv2.putText(blank_image_original, 'Original', (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
    cv2.putText(blank_image_predicted, 'Predicted', (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    # Concatenate the two images side by side
    comparison_image = np.hstack((blank_image_original, blank_image_predicted))

    # Display the side-by-side comparison
    cv2.imshow('Original vs Predicted Posture', comparison_image)

    # Wait for keypress or a delay between frames
    key = cv2.waitKey(50000) & 0xFF  # Adjust the delay as needed
    if key == ord('q'):
        print("Exiting on 'q' key press")
        break

# Release resources
cv2.destroyAllWindows()


C:\Users\Test\AppData\Local\Temp\ipykernel_19416\2052921703.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  x = predicted_row[i] * width  # Assuming x is normalized, scale to image width
C:\Users\Test\AppData\Local\Temp\ipykernel_19416\2052921703.py:40: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  y = predicted_row[i + 1] * height  # Assuming y is normalized, scale to image height
C:\Users\Test\AppData\Local\Temp\ipykernel_19416\2052921703.py:47: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFra

Exiting on 'q' key press


C:\Users\Test\AppData\Local\Temp\ipykernel_19416\2052921703.py:129: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  x = predicted_row[i] * width  # Assuming x is normalized, scale to image width
C:\Users\Test\AppData\Local\Temp\ipykernel_19416\2052921703.py:130: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  y = predicted_row[i + 1] * height  # Assuming y is normalized, scale to image height
C:\Users\Test\AppData\Local\Temp\ipykernel_19416\2052921703.py:137: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with Data